In [1]:
import os
import pdf2image
from PIL import Image
import pytesseract
import difflib
import re
import pandas as pd
import sys
sys.path.append('../')
from helper import *

In [2]:
DATA_PATH = '../../data/'
STATE = 'lakshadweep/'

PARSE_DATA_PAGES = "../../parseData/images/"+STATE
create_path(PARSE_DATA_PAGES)

PARSE_DATA_BLOCKS = "../../parseData/blocks/"+STATE
create_path(PARSE_DATA_BLOCKS)

PARSE_DATA_CSVS = "../../parseData/csvs/"+STATE
create_path(PARSE_DATA_CSVS)

COLUMNS = ["number","id", "elector_name", "father_or_husband_name", "relationship", "house_no", "age", "sex", "ac_name", "parl_constituency", "part_no", "year", "state", "filename", "main_town", "police_station", "mandal", "revenue_division", "district", "pin_code", "polling_station_name", "polling_station_address", "net_electors_male", "net_electors_female", "net_electors_third_gender", "net_electors_total"]

state_pdfs_path = DATA_PATH+STATE
state_pdfs_files = os.listdir(state_pdfs_path)

In [3]:
sort_nicely(state_pdfs_files)
state_pdfs_files

['001001.pdf',
 '001002.pdf',
 '001003.pdf',
 '001004.pdf',
 '001005.pdf',
 '001006.pdf',
 '001007.pdf',
 '001008.pdf',
 '001009.pdf',
 '001010.pdf',
 '001011.pdf',
 '001012.pdf',
 '001013.pdf',
 '001014.pdf',
 '001015.pdf',
 '001016.pdf',
 '001017.pdf',
 '001018.pdf',
 '001019.pdf',
 '001020.pdf',
 '001021.pdf',
 '001022.pdf',
 '001023.pdf',
 '001024.pdf',
 '001025.pdf',
 '001026.pdf',
 '001027.pdf',
 '001028.pdf',
 '001029.pdf',
 '001030.pdf',
 '001031.pdf',
 '001032.pdf',
 '001033.pdf',
 '001034.pdf',
 '001035.pdf',
 '001036.pdf',
 '001037.pdf',
 '001038.pdf',
 '001039.pdf',
 '001040.pdf',
 '001041.pdf',
 '001042.pdf',
 '001043.pdf',
 '001044.pdf',
 '001045.pdf',
 '001046.pdf',
 '001047.pdf',
 '001048.pdf']

In [5]:
def split_data(data, seps):
    
    for s in seps:
        
        if s in data:
            break
    
    data = data.split(s)

    if len(data)>1:
        data = data[-1].strip()
        return data
    else:
        data = ""
        return data
    
def extract_first_page_details(path):

    img = Image.open(path)
    crop_path = input_images_blocks_path+"page/"
    create_path(crop_path)
    
    a,b,c,d = 3476,472,269,120 # part no
    crop_img = crop_section(a,b,c,d,img)
    
    crop_part_path = crop_path+"part.jpg"
    crop_img.save(crop_part_path)

    text = (pytesseract.image_to_string(crop_part_path, config='--psm 6', lang='eng+pan')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = re.findall(r'\d+', text)
    
    if len(text)>0:
        part_no = text[0]
    else:
        part_no = ""
        
    
    a,b,c,d = 204,692,2546,261 # ac name and parl
    crop_img = crop_section(a,b,c,d,img)
    
    crop_ac_path = crop_path+"ac.jpg"
    crop_img.save(crop_ac_path)

    text = (pytesseract.image_to_string(crop_ac_path, config='--psm 6', lang='eng+mal')) #config='--psm 4' config='-c preserve_interword_spaces=1'
    text = text.split('\n')
    text = [ i for i in text if i!='' and i!='\x0c']
    
    
    ac_name, parl_constituency = "",""
    seps = ["ലോകസഭാമണ്ഡലത്തിന്റെ"]
    if len(text) == 3:
        ac_name = split_data(text[1],seps)
    
    
    polling_station_name, polling_station_address = "", ""

    main_town,police_station,revenue_division,mandal,district,pin_code,a_n,b_n,c_n,d_n = "","","","","","","","","",""

   
    return [ac_name,parl_constituency,part_no,main_town,police_station,polling_station_name,polling_station_address,revenue_division,mandal,district,pin_code,a_n,b_n,c_n,d_n]


def arrange_columns(first_page_list,block_list,filename):
    
    year = 2017
    state = 'lakshadweep'
    
    ac_name,parl_constituency,part_no,main_town,police_station,polling_station_name,polling_station_address,revenue_division,mandal,district,pin_code,net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total = first_page_list
    name,rel_name,rel_type,house_no,age,gender,voter_id,number = block_list
    
    final_list = [number,voter_id,name,rel_name,rel_type,house_no,age,gender,ac_name,
                 parl_constituency,part_no,year,state,filename,main_town,police_station,mandal,
                 revenue_division,district,pin_code,polling_station_name,polling_station_address,
                 net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total]
    
    
    return final_list


In [17]:
def generate_poll_blocks_from_page(page_full_path,page_blocks_path,amend_page):
    
    img = Image.open(page_full_path)
    
    amend = False
    
    def generate(intial_width,a,b,gap):
        count = 0
        crop_width = 1215
        crop_height = 420

        for col in range(1,11):

            for row in range(1,4):
                c = a+crop_width
                d = b+crop_height
                area = (a, b, c, d)
                cropped_img = img.crop(area)
                count = count+1
                
                new_area = (900,100, 1300, 470)
                
                region = Image.new("RGB", (400, 370), (255, 255, 255))
                cropped_img.paste(region,new_area)
                
                cropped_img.save(page_blocks_path+str(count)+".jpg")

                a = c

            a = intial_width
            b = b+crop_height+gap
    
    page_type,intial_height = check_page_type(img,amend_page)
    
    if page_type == 1:
        intial_width = 205
        generate(intial_width,intial_width,intial_height,3)
        amend_page = False
   
    return amend_page

def check_page_type(img,amend_page):
    
    return 1,408
    

In [51]:
def extract_name(name):
    
    seps = ["പേര്"," "]
    name = split_data(name, seps)
            
    
    return name
    
def extract_vid(v_id):
    
    seps = ["|", "_","~","=","-","}"]
    for s in seps:
        v_id = v_id.replace(s,"")
    
    row = v_id.split(" ")
    
    if len(row) == 1:
        return "",row[0]
    elif len(row)==2:
        number = re.findall(r'\d+', row[0].strip())
        if len(number)>0:
            return number[0],row[1]
        else:
            return "",row[1]
    
    elif len(row)>2:
        
        number = re.findall(r'\d+', row[0].strip())
        if len(number)>0:
            return number[0],row[1]+" "+row[2]
        else:
            return "",row[1]+" "+row[2]  
    else:
        return "",v_id

def extract_house_no(house_no):
    seps = ["വീട്ടുനമ്പര്"]
    house_no = split_data(house_no,seps)
         
    return house_no
    
def extract_age_gender(age_gender):
    
    age =  re.findall(r'\d+', age_gender)
    
    if len(age)>0:
        age =  age[0]
    else:
        age = ""
    
    if age == "":
        row = age_gender.split("/")
        if len(row)>1:
            age =  row[-1]
    
    
    if "സ്ത്രീ" in age_gender:
        gender = "Female"
    else:
        gender = "Male"
   

    return age, gender

def extract_rel_name(rel_name):
    
    seps = ["പേര്"]
    
    name = split_data(rel_name, seps)
    rel_type = extract_rel_type(rel_name)
           
    return name,rel_type
    
def extract_rel_type(rel_type):

    if "അച്ഛന്" in rel_type:
        rel_type = 'father'
        
    elif "അമ്മയുടെ" in rel_type:
        rel_type = 'mother'
        
    elif "dത്താവിന്റെ" in rel_type:
        rel_type = 'husband'
        
    else:
        rel_type = "father"
    
    return rel_type 


def extract_details_from_block(block):
    
    number,voter_id = "",""
    
    if len(block) == 5:
        name = block[0]
        rel_name = block[1]
        house_no = block[2]
        age_gender = block[4]
        
    else:    
        v_id = block[0]
        name = block[1]
        rel_name = block[2]
        house_no = block[3]
        age_gender = block[4]
        voter_id = v_id

    name = extract_name(name)
    rel_name,rel_type = extract_rel_name(rel_name)
    house_no = extract_house_no(house_no)
    age, gender = extract_age_gender(age_gender)
    
    return [name,rel_name,rel_type,house_no,age,gender,voter_id,number]


In [54]:
# for pdf_file_name in ['001002.pdf']:
for pdf_file_name in state_pdfs_files:
    
    print(pdf_file_name)
    if pdf_file_name == ".DS_Store":
        continue
    
    if not pdf_file_name.endswith(".pdf"):
        continue
    
    #create images,blocks and csvs paths for each file
    pdf_file_name_without_ext = pdf_file_name.split('.pdf')[0]
    input_pdf_images_path = PARSE_DATA_PAGES+pdf_file_name_without_ext+"/"
    create_path(input_pdf_images_path)

    input_images_blocks_path = PARSE_DATA_BLOCKS+pdf_file_name_without_ext+"/"
    create_path(input_images_blocks_path)
    
    if os.path.exists(PARSE_DATA_CSVS+pdf_file_name_without_ext+".csv"):
        print(pdf_file_name_without_ext+".csv", "already exists")
        continue
    
    #convert pdf into bunch of images
    pdf_2_images_list = pdf_to_img(state_pdfs_path+pdf_file_name, input_pdf_images_path,dpi=500)
    
    
    #sort pages for looping
    input_images = os.listdir(input_pdf_images_path)
    sort_nicely(input_images)
    
    #empty intial data
    df = pd.DataFrame(columns = COLUMNS)
    order_problem = []
    
    amend_page = False
    
    #for each page, parse the data
    for page in input_images:
    
        page_full_path = input_pdf_images_path+page
        
        #extract first page content
        if page == '1.jpg':
            first_page_list = extract_first_page_details(page_full_path)
            continue
            

        #ingnore 2nd page and last page
        if page == '2.jpg' or input_images[-1] == page:
            continue
                    

        #loop from 3 page onwards
        if page.endswith('.jpg'):
            
            final_invidual_blocks = []
            blocks_path = input_images_blocks_path+"blocks/"
            create_path(blocks_path)

            page_idx = page.split(".jpg")[0] + "/"
            page_blocks_path = blocks_path+page_idx
            create_path(page_blocks_path)
                
            amend_page = generate_poll_blocks_from_page(page_full_path,page_blocks_path,amend_page)
            
            
            sorted_blocks = os.listdir(page_blocks_path)
            sort_nicely(sorted_blocks)
            
            for jpg_file in sorted_blocks:
                
                if jpg_file.endswith('.jpg') :

                    text = (pytesseract.image_to_string(page_blocks_path+jpg_file, config='--psm 6', lang='eng+mal')) #config='--psm 4' config='-c preserve_interword_spaces=1'
                    params_list = text.split('\n')
                    new_params_list = [ i for i in params_list if i!='' and i!='\x0c']
                    
                    if len(new_params_list) ==5 or len(new_params_list) == 6:
                        if len(new_params_list) == 6 and "പു" in new_params_list[0]:
                            order_problem.append((page, jpg_file,new_params_list))
                        else:
                            final_invidual_blocks.append(new_params_list)
                    else:
                        order_problem.append((page, jpg_file,new_params_list))
        
        #put the data into dataframe
        for block in final_invidual_blocks:
            block_list = extract_details_from_block(block)
            
            final_list = arrange_columns(first_page_list,block_list,pdf_file_name_without_ext)
            
            df_length = len(df)
            df.loc[df_length] = final_list
        
        print("page done : ",page)
                
    #save the dataframe(pdf) data into csv
    save_to_csv(df,PARSE_DATA_CSVS+pdf_file_name_without_ext+".csv")
    print("CSV saved")
    print()


001002.pdf
page done :  3.jpg
page done :  4.jpg
page done :  5.jpg
page done :  6.jpg
page done :  7.jpg
page done :  8.jpg
page done :  9.jpg
page done :  10.jpg
page done :  11.jpg
page done :  12.jpg
page done :  13.jpg
page done :  14.jpg
page done :  15.jpg
page done :  16.jpg
page done :  17.jpg
page done :  18.jpg
page done :  19.jpg
page done :  20.jpg
page done :  21.jpg
page done :  22.jpg
page done :  23.jpg
page done :  24.jpg
page done :  25.jpg
page done :  26.jpg
page done :  27.jpg
page done :  28.jpg
page done :  29.jpg
page done :  30.jpg
CSV saved



In [ ]:
#combine all state files into one csv
combine_all_csvs("lakshadweep_final.csv",PARSE_DATA_CSVS)